In [6]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [7]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/

# This requires you to checkout https://github.com/peterhinch/micropython-async
%sendtofile --source /home/julian/extrepositories/micropython-async/asyn.py
%sendtofile --source /home/julian/extrepositories/micropython-async/aswitch.py

# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py
%sendtofile --source ../Sensor_Kennel/sevensegmentdisplay.py


Sent 79 lines (2294 bytes) to sevensegmentdisplay.py.


In [3]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [8]:
# Code is at jupyter_micropython_developer_notebooks/projects/IRtemp_to_mqtt.ipynb
from sevensegmentdisplay import encodeledstring
from machine import I2C, Pin
import time
from sevensegmentdisplay import encodeledstring, writeledstringautoscroll

encodeledstring("start up")


In [19]:
import time
print(time.ticks_ms())

11484


In [25]:
from sevensegmentdisplay import encodeledstring, writeledstringautoscroll
encodeledstring(b" dog onn")
writeledstringautoscroll()


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [44]:
%sendtofile main.py

# Code is at jupyter_micropython_developer_notebooks/projects/IRtemp_to_mqtt.ipynb
from sevensegmentdisplay import encodeledstring, writeledstringautoscroll
import machine, ubinascii, time

wifiname = "DoESLiverpool"
wifipassword = "decafbad00"
mqttbroker = "mqtt.local"
boardname = "whitesp"
watchdogtimeoutseconds = 60

encodeledstring("start up")
time.sleep(5)

# from here on the watchdog will reset if there are any crashing out
w = machine.WDT(timeout=watchdogtimeoutseconds*1000)
encodeledstring(" dog on")
time.sleep(2)

resetcause = { machine.PWRON_RESET:"PWRON_RESET", machine.HARD_RESET:"HARD_RESET", machine.WDT_RESET:"WDT_RESET", machine.DEEPSLEEP_RESET:"DEEPSLEEP_RESET", machine.SOFT_RESET:"SOFT_RESET"}\
    .get(machine.reset_cause(), "RESET_%d"%machine.reset_cause())


i2c = machine.I2C(scl=machine.Pin(15), sda=machine.Pin(4), freq=100000)

from mqtt_as import MQTTClient
import uasyncio as asyncio
import asyn
from mqtt_as import config

tempAmbient, tempIR = 0, 0
tempBad = True
tempTime = 0

def checkCRC(k, mask):  # mask (matching nibble) different in case
    inCrc = 0
    for inData in [k[0], k[1]]:
        data = inCrc ^ inData;
        data <<= 8 
        for i in range(8):
            if (( data & 0x8000 ) != 0 ):
                data = data ^ (0x1070 << 3)
            data = data << 1
        inCrc = data >> 8
    return (inCrc & mask) == (k[2]&mask)

timeholdmessage = 0
def holdmessage(msg, durationseconds=10):
    global timeholdmessage
    timeholdmessage = time.ticks_ms() + durationseconds*1000
    encodeledstring(msg)

async def readdisplaytemps():
    global ntempcount
    while True:
        global tempAmbient, tempIR, tempBad, tempTime
        try:
            ka = i2c.readfrom_mem(0x5a, 0x06, 3)
            ki = i2c.readfrom_mem(0x5a, 0x07, 3)
            bcrcgood = (checkCRC(ka, 0x0F) and checkCRC(ki, 0xF0))
            print(bcrcgood, ka, ki)
            if not bcrcgood:
                raise OSError()
            tempAmbient = (ka[0]+ka[1]*256)*0.02 - 273.15
            tempIR = (ki[0]+ki[1]*256)*0.02 - 273.15
            tempBad = False
        except OSError:
            tempBad = True
        tempTime = time.ticks_ms()

        if time.ticks_ms() > timeholdmessage:
            if not tempBad:
                encodeledstring("%5.2f %.1f"% (tempIR, tempAmbient))
            else:
                encodeledstring("bad %04d"%(tempTime%10000))
        else:
            writeledstringautoscroll()

        await asyncio.sleep_ms(200)

def callback(topic, msg):
    print("callback", topic, msg)
    holdmessage(msg, durationseconds=10)
        
config['server'] = mqttbroker
config['ssid'] = wifiname
config['wifi_pw'] = wifipassword
config['mqttchannel'] = boardname
config['subs_cb'] = callback

mqttchannelA = boardname+'/tempambient'
mqttchannelI = boardname+'/tempir'
mqttchannelIP = boardname+'/ip'
mqttchannelBad = boardname+'/tempoutofrange'

async def main(client):
    DtempTime = 0
    await client.connect()
    holdmessage("connected")
    w.feed()
    await client.subscribe(boardname+'/msg', 1)
        
    ipnumber = client._sta_if.ifconfig()[0]
    macaddress = ubinascii.hexlify(client._sta_if.config('mac'),':').decode().upper()

    await client.publish(boardname+'/ip', ipnumber, retain=True, qos=1)
    await client.publish(boardname+'/mac', macaddress, retain=True, qos=1)
    await client.publish(boardname+"/resetcause", resetcause, retain=True, qos=1)
    w.feed()
    await asyncio.sleep_ms(2000)
        
    n = 0
    while True:
        w.feed()
        if DtempTime != tempTime:
            if not tempBad:  # no inconsistency problem on this as async
                if -10<tempAmbient<40:
                    await client.publish(mqttchannelA, '%.2f'%tempAmbient, qos=1)
                else:
                    await client.publish(mqttchannelBad, 'A %.2f'%tempAmbient, qos=1)

            if not tempBad:  # no inconsistency problem on this as async
                if -10<tempIR<40:
                    await client.publish(mqttchannelI, '%.2f'%tempIR, qos=1)
                else:
                    await client.publish(mqttchannelBad, 'I %.2f'%tempIR, qos=1)
            DtempTime = tempTime
        await asyncio.sleep_ms(100)


MQTTClient.DEBUG = True   # Optional: print diagnostic messages
client = MQTTClient(config)
aloop = asyncio.get_event_loop()
aloop.create_task(readdisplaytemps())
w.feed()

encodeledstring("ready.")
msg = "fail."
try:
    aloop.run_until_complete(main(client))
except Exception as e:
    print(e)
    msg = str(e).lower()
client.close()  # Prevent LmacRxBlk:1 errors

encodeledstring(msg)
while 1:
    writeledstringautoscroll()


Sent 147 lines (4667 bytes) to main.py.
